# Tutorial: From a CNN model to a DYNAP-CNN configuration

This tutorial explains all steps necessary to convert a torch CNN model to a configuration of the DYNAP-CNN chip. We will first convert the network to a spiking neural network (SNN) and then to a `DynapcnnCompatibleNetwork` – a model that is compatible with the chip and simulates its behavior. Finally we generate a configuration object that can be used to set up the model on the hardware.

## Import libraries

Before we start, we will import the libraries necessary to define a torch model, convert it to an SNN and to convert the SNN to a `DynapcnnCompatibleNetwork`.

In [1]:
# - Import statements

from torch import nn, rand
from sinabs.from_torch import from_model
from sinabs.backend.dynapcnn import DynapcnnCompatibleNetwork

## CNN definition

First we will define a sequential CNN model. To see what SINABS is capable of, we include elements like Dropout and batch normalization.

*Note that although non-sequential models are supported by the hardware, this is not yet the case for this library.*

In [2]:
# - Define CNN model

class MyCNN(nn.Module):
    def __init__(self, n_channels_in=4, n_channels_out=1):
        
        super().__init__()
        
        seq = [
            # Convolutional layer with padding and bias
            nn.Conv2d(
                in_channels=n_channels_in,
                out_channels=12,
                kernel_size=(4,4),
                padding=(1,1),
                bias=True,
            ),
            # ReLU
            nn.ReLU(),
            # Average pooling (stride equal to kernel size)
            nn.AvgPool2d(kernel_size=(2,2), stride=(2,2)),
            # Convolutional layer and batch normalization
            nn.Conv2d(
                in_channels=12,
                out_channels=16,
                kernel_size=(3,3),
                bias=True,
            ),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(4,4), stride=(4,4)),
            # Dropout
            nn.Dropout2d(0.5),
            # Two fully connected layers
            nn.Flatten(),
            nn.Linear(784, 32, bias=False),
            nn.ReLU(),
            nn.Linear(32, n_channels_out, bias=True),
            nn.ReLU(),
        ]
        self.seq = nn.Sequential(*seq)
        
    def forward(self, x):
        return self.seq(x)
    
cnn = MyCNN()
print(cnn)

MyCNN(
  (seq): Sequential(
    (0): Conv2d(4, 12, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    (3): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): AvgPool2d(kernel_size=(4, 4), stride=(4, 4), padding=0)
    (7): Dropout2d(p=0.5, inplace=False)
    (8): Flatten()
    (9): Linear(in_features=784, out_features=32, bias=False)
    (10): ReLU()
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): ReLU()
  )
)


## Convert to Spiking CNN

We can use the `from_torch` method from SINABS to convert our CNN to a SNN. The returned object contains the original CNN as `analog_model` and the newly generated SNN as `spiking_model`. The `ReLU`s have been converted to `SpikingLayers`.

In [3]:
snn = from_model(cnn)
print(snn.spiking_model)

MyCNN(
  (seq): Sequential(
    (0): Conv2d(4, 12, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (1): SpikingLayer()
    (2): AvgPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0)
    (3): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): SpikingLayer()
    (6): AvgPool2d(kernel_size=(4, 4), stride=(4, 4), padding=0)
    (7): Dropout2d(p=0.5, inplace=False)
    (8): Flatten()
    (9): Linear(in_features=784, out_features=32, bias=False)
    (10): SpikingLayer()
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): SpikingLayer()
  )
)


## DYNAP-CNN compatible network

The next step is to convert the SNN to a `DynapcnnCompatibleNetwork`. This way we can be sure that all functionalities of our network are supported by the hardware and we can simulate the expected hardware output for testing purposes. This object will also generate the configuration objects to set up the chip.

We need to tell the chip the dimensions of the input data. This can be done either by specifying an `input_shape` argument in the constructor or including a SINABS `InputLayer` at the beginning of the model.

The class will convert the parameters (weights, biases, and thresholds) to discrete values that are supported by DYNAP-CNN. For testing purposes this can be disabled by setting `discretize` to `False`.

We can use the `dvs_input` flag to determine whether the chip should process data coming from the on-chip dynamic vision sensor (DVS).

In [4]:
# - Input dimensions
input_shape = (4, 64, 64)

# - DYNAP-CNN compatible network
dynapcnn_net = DynapcnnCompatibleNetwork(
    snn,
    input_shape=input_shape,
    discretize=False,
    dvs_input=True,
)
print(dynapcnn_net)

DynapcnnCompatibleNetwork(
  (sequence): Sequential(
    (0): DynapcnnLayer(
      (_conv_layer): Conv2d(4, 12, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=2, stride=2, ceil_mode=False)
      (_spk_layer): SpikingLayer()
    )
    (1): DynapcnnLayer(
      (_conv_layer): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1))
      (_pool_layer): SumPool2d(norm_type=1, kernel_size=4, stride=4, ceil_mode=False)
      (_spk_layer): SpikingLayer()
    )
    (2): DynapcnnLayer(
      (_conv_layer): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), bias=False)
      (_spk_layer): SpikingLayer()
    )
    (3): DynapcnnLayer(
      (_conv_layer): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1))
      (_spk_layer): SpikingLayer()
    )
  )
)


The resulting model consists of four `DynapcnnLayer` objects, each containing a convolutional, a spiking, and possibly a pooling layer.

In [5]:
# - Run network on random data
input_data = rand((1, *input_shape)) * 1000
output_data = dynapcnn_net(input_data)

# - Model is quantized, so the output will have an integer value.
print(output_data)

tensor([[[[9.]]]])


## DYNAP-CNN configuration

We can now extract a dynapcnn configuration object from the `DynapcnnCompatibleNetwork`, which can then be used to configure the hardware. In order to map layers of the sequential model to specific layers on the chip we can provide a list of layer indices in the order of the data flow.

In [6]:
config = dynapcnn_net.make_config(dynapcnn_layers_ordering=[4, 2, 1, 7])

## Validation and upload to DYNAP-CNN

Finally, we only need to upload the configuration to the hardware. Before that however, it should be made sure that the way the layers are arranged is compatible.

The functionalities for validation and uploading will soon be added and explained here.